#Local Extrema

Investigate the following function for local extrema:
$$z = e^{x-y} (x^2 - 2y^2)$$

In [1]:
import sympy as sp

def analyze_function(expr, vars):
    print("Function:")
    sp.pprint(expr)

    # Gradient and critical points
    grad = [sp.diff(expr, v) for v in vars]
    crit_all = sp.solve(grad, vars, dict=True)

    # Keep only real critical points
    crit = []
    for c in crit_all:
        if all(v.is_real for v in c.values()):
            crit.append(c)

    print("\nCritical points:")
    if not crit:
        print("  None")
    else:
        for c in crit:
            print(" ", tuple(c[v] for v in vars))

    # Hessian & classification
    H = sp.hessian(expr, vars)
    print("\nHessian:")
    sp.pprint(H)

    local_min = []
    local_max = []
    saddle = []

    for c in crit:
        Hc = H.subs(c)
        eigs = list(Hc.eigenvals().keys())

        pos = all(ev.is_real and ev.is_positive for ev in eigs)
        neg = all(ev.is_real and ev.is_negative for ev in eigs)

        if pos:
            local_min.append(c)
        elif neg:
            local_max.append(c)
        else:
            saddle.append(c)

    print("\nLocal extrema:")

    print("Local maxima:")
    if not local_max:
        print("  (no local maxima found)")
    else:
        for c in local_max:
            print(" ", tuple(c[v] for v in vars))

    print("\nLocal minima:")
    if not local_min:
        print("  (no local minima found)")
    else:
        for c in local_min:
            val = sp.simplify(expr.subs(c))
            print(f"  min(f) = {val} at {tuple(c[v] for v in vars)}")

    # Global extrema
    print("\nGlobal extrema:")
    print("Global maxima")
    print("  (no global maxima found)")

    if local_min:
        vals = [(sp.simplify(expr.subs(c)), c) for c in local_min]
        best_val = min(vals, key=lambda x: x[0])[0]
        best_pts = [c for v, c in vals if v == best_val]

        print("\nGlobal minima:")
        for c in best_pts:
            print(f"  min(f) = {best_val} at (x, y) = "
                  f"({sp.simplify(c[vars[0]])}, {sp.simplify(c[vars[1]])})")
    else:
        print("\nGlobal minima:")
        print("  (no global minima found)")

In [9]:
x, y = sp.symbols('x y')
f = sp.exp(x - y) * (x**2 - 2*y**2)
#f = x**4 + y**4 - 2*x**2 -4*x*y -2*y**2
analyze_function(f, [x, y])

# Critical points
grad = [sp.diff(f, x), sp.diff(f, y)]
crit_all = sp.solve(grad, (x, y), dict=True)

Function:
⎛ 2      2⎞  x - y
⎝x  - 2⋅y ⎠⋅ℯ     

Critical points:
  (-4, -2)
  (0, 0)

Hessian:
⎡       x - y   ⎛ 2      2⎞  x - y      x - y           x - y        x - y   ⎛ ↪
⎢  4⋅x⋅ℯ      + ⎝x  - 2⋅y ⎠⋅ℯ      + 2⋅ℯ         - 2⋅x⋅ℯ      - 4⋅y⋅ℯ      - ⎝ ↪
⎢                                                                              ↪
⎢       x - y        x - y   ⎛ 2      2⎞  x - y         x - y   ⎛ 2      2⎞  x ↪
⎣- 2⋅x⋅ℯ      - 4⋅y⋅ℯ      - ⎝x  - 2⋅y ⎠⋅ℯ         8⋅y⋅ℯ      + ⎝x  - 2⋅y ⎠⋅ℯ  ↪

↪  2      2⎞  x - y⎤
↪ x  - 2⋅y ⎠⋅ℯ     ⎥
↪                  ⎥
↪  - y      x - y  ⎥
↪      - 4⋅ℯ       ⎦

Local extrema:
Local maxima:
  (-4, -2)

Local minima:
  (no local minima found)

Global extrema:
Global maxima
  (no global maxima found)

Global minima:
  (no global minima found)


# Conditional Extrema
Investigate the following function for global extrema
$$z(x, y) = a\cos^2x + b\cos^2y$$
given the condition $y - x = \frac{\pi}{4}$

In [11]:
import sympy as sp

def extrema_single_constraint(f, vars, constraints):
    # Solve constraint for one variable
    sol = sp.solve(constraints, vars[1], dict=True)[0]

    # Substitute back: reduce f(x,y) -> f(x)
    f_reduced = sp.simplify(f.subs(sol))

    # 1-variable function in vars[0]
    x = vars[0]

    # First derivative
    df = sp.diff(f_reduced, x)

    # Critical points
    crit_points = sp.solve(df, x)

    result = []

    for cp in crit_points:
        val = sp.simplify(f_reduced.subs(x, cp))
        result.append((cp, val))

    return f_reduced, result

x, y, a, b = sp.symbols('x y a b', real=True)

f = a*sp.cos(x)**2 + b*sp.cos(y)**2
constraint = [y - x - sp.pi/4]

f_reduced, critical_points = extrema_single_constraint(f, [x, y], constraint)

print("Reduced 1-variable function f(x):")
sp.pprint(f_reduced)

print("\nCritical points and function values:")
for cp, val in critical_points:
    print("\n x =", cp)
    print(" f =", val)

Reduced 1-variable function f(x):
     2           2⎛    π⎞
a⋅cos (x) + b⋅cos ⎜x + ─⎟
                  ⎝    4⎠

Critical points and function values:

 x = -2*atan(-a/b + sqrt(b**2 + (a - sqrt(a**2 + b**2))**2)/Abs(b) + sqrt(a**2 + b**2)/b)
 f = a*cos(2*atan(-a/b + sqrt(b**2 + (a - sqrt(a**2 + b**2))**2)/Abs(b) + sqrt(a**2 + b**2)/b))**2 + b*sin(2*atan(-a/b + sqrt(b**2 + (a - sqrt(a**2 + b**2))**2)/Abs(b) + sqrt(a**2 + b**2)/b) + pi/4)**2

 x = 2*atan(a/b + sqrt(b**2 + (a - sqrt(a**2 + b**2))**2)/Abs(b) - sqrt(a**2 + b**2)/b)
 f = a*cos(2*atan(a/b + sqrt(b**2 + (a - sqrt(a**2 + b**2))**2)/Abs(b) - sqrt(a**2 + b**2)/b))**2 + b*cos(2*atan(a/b + sqrt(b**2 + (a - sqrt(a**2 + b**2))**2)/Abs(b) - sqrt(a**2 + b**2)/b) + pi/4)**2

 x = 2*atan(a/b - sqrt(b**2 + (a + sqrt(a**2 + b**2))**2)/Abs(b) + sqrt(a**2 + b**2)/b)
 f = a*cos(2*atan(a/b - sqrt(b**2 + (a + sqrt(a**2 + b**2))**2)/Abs(b) + sqrt(a**2 + b**2)/b))**2 + b*cos(2*atan(a/b - sqrt(b**2 + (a + sqrt(a**2 + b**2))**2)/Abs(b) + sqrt(a**2 +

Investigate the following function for global extrema
$$u(x, y, z) = xyz$$
given the conditions:

1.   $x + y + z = 5$
2.   $xy + xz + yz = 8$



In [12]:
import sympy as sp

def extrema_with_constraints(f, vars, constraints):

    sol = sp.solve(constraints, vars, dict=True)

    if not sol:
        raise ValueError("Constraints cannot be solved.")

    results = []

    for s in sol:
        # substitute constraints -> reduced function
        f_reduced = sp.simplify(f.subs(s))

        # find critical points of reduced function
        free_vars = [v for v in vars if v not in s]

        if not free_vars:
            # No variables left -> value is constant
            f_val = sp.simplify(f_reduced)
            results.append(("constant", s, f_val))
            continue

        grad_eqs = [sp.diff(f_reduced, v) for v in free_vars]
        crit_solutions = sp.solve(grad_eqs, free_vars, dict=True)

        # evaluate f on each critical point
        for c in crit_solutions:
            point = s.copy()
            point.update(c)
            value = sp.simplify(f.subs(point))
            results.append((point, value))

    # classify extrema
    numeric_vals = []
    for p, v in results:
        if p == "constant":
            numeric_vals.append(v)
        else:
            numeric_vals.append(sp.N(v))

    f_min = min(numeric_vals)
    f_max = max(numeric_vals)

    global_min = [(p, v) for p, v in results if sp.N(v) == f_min]
    global_max = [(p, v) for p, v in results if sp.N(v) == f_max]

    return {
        "global_min": global_min,
        "global_max": global_max,
        "critical_points": results
    }

In [13]:
x, y, z = sp.symbols('x y z', real=True)

u = x*y*z

constraints = [
    x + y + z - 5,
    x*y + x*z + y*z - 8
]

result = extrema_with_constraints(u, [x, y, z], constraints)

print("Critical points:")
for p, v in result["critical_points"]:
    print(" point =", p, "   value =", sp.simplify(v))

print("\nGlobal minima:")
for p, v in result["global_min"]:
    print(" point =", p, "   value =", sp.simplify(v))

print("\Global maxima:")
for p, v in result["global_max"]:
    print(" point =", p, "   value =", sp.simplify(v))

Critical points:
 point = {x: -z/2 - sqrt(-(z - 1)*(3*z - 7))/2 + 5/2, y: -z/2 + sqrt(-(z - 1)*(3*z - 7))/2 + 5/2, z: 4/3}    value = 112/27
 point = {x: -z/2 - sqrt(-(z - 1)*(3*z - 7))/2 + 5/2, y: -z/2 + sqrt(-(z - 1)*(3*z - 7))/2 + 5/2, z: 2}    value = 4
 point = {x: -z/2 + sqrt(-(z - 1)*(3*z - 7))/2 + 5/2, y: -z/2 - sqrt(-(z - 1)*(3*z - 7))/2 + 5/2, z: 4/3}    value = 112/27
 point = {x: -z/2 + sqrt(-(z - 1)*(3*z - 7))/2 + 5/2, y: -z/2 - sqrt(-(z - 1)*(3*z - 7))/2 + 5/2, z: 2}    value = 4

GLOBAL MINIMA:
 point = {x: -z/2 - sqrt(-(z - 1)*(3*z - 7))/2 + 5/2, y: -z/2 + sqrt(-(z - 1)*(3*z - 7))/2 + 5/2, z: 2}    value = 4
 point = {x: -z/2 + sqrt(-(z - 1)*(3*z - 7))/2 + 5/2, y: -z/2 - sqrt(-(z - 1)*(3*z - 7))/2 + 5/2, z: 2}    value = 4

GLOBAL MAXIMA:
 point = {x: -z/2 - sqrt(-(z - 1)*(3*z - 7))/2 + 5/2, y: -z/2 + sqrt(-(z - 1)*(3*z - 7))/2 + 5/2, z: 4/3}    value = 112/27
 point = {x: -z/2 + sqrt(-(z - 1)*(3*z - 7))/2 + 5/2, y: -z/2 - sqrt(-(z - 1)*(3*z - 7))/2 + 5/2, z: 4/3}    va